## Результаты


Значение коэффициента корреляции Спирмена между человеческими оценками и косинусной близостью между векторами Fasttext:
    
    - Для списка similarity: 83.19;
    - Для списка relatedness: 73.1.

**Вывод**: Значения косинусной близости между векторами модели Fasttext хорошо коррелируют с человеческими оценками сходства между словами. Следовательно, схожие по значениям слова находятся близко друг к другу в пространстве векторов, а сами вектора слов информативны с точки зрения семантики. 

In [2]:
# !pip -q install fasttext
# !pip show scipy

In [3]:
import fasttext
import fasttext.util
from scipy.spatial.distance import cosine
from scipy.stats import spearmanr

In [4]:
def load_similarity_triplets(inp_path: str, sep='\t'):
    word_similarity_triples: List[Tuple[str, str, float]] = []
    with open(inp_path, 'r', encoding="utf-8") as inp_file:
        for line in inp_file:
            attrs = line.strip().split(sep)
            word_1 = attrs[0].lower()
            word_2 = attrs[1].lower()
            similarity = float(attrs[2])
            word_similarity_triples.append((word_1, word_2, similarity))
    return word_similarity_triples

In [5]:
relatedness_file_path = "./wordsim353_sim_rel/wordsim_relatedness_goldstandard.txt"
similarity_file_path = "./wordsim353_sim_rel/wordsim_similarity_goldstandard.txt"

In [6]:
relatedness_triples = load_similarity_triplets(inp_path=relatedness_file_path)
similarity_triples = load_similarity_triplets(inp_path=similarity_file_path)

### Загрузка предобученной модели Fasttext

In [7]:
# fasttext.util.download_model('en',) 
ft_model = fasttext.load_model('cc.en.300.bin')

File exists. Use --overwrite to download anyway.


### Создание списка человеческих оценок relatedness и similarity

In [9]:
human_relatedness = [t[2] for t in relatedness_triples]
human_similarities = [t[2] for t in similarity_triples]

In [10]:
def calculate_fasttext_pairwise_cos_sims(fasttext_model, word_pair_similarities):
    # Функция получает на вход список пар слов, для обоих слов каждый пары находит соответствующий вектор
    # с помощью модели Fasttext. После чего находит значение косинусной близости между парами этих векторов.
    # Функция возвращает список значений косинусной близости для всех пар слов из входного списка
    fasttext_similarities = []
    for (w_1, w_2, _) in word_pair_similarities:
        w_1_ft_vector = fasttext_model[w_1]
        w_2_ft_vector = fasttext_model[w_2]
        cosine_sim = 1. - cosine(w_1_ft_vector, w_2_ft_vector)
        fasttext_similarities.append(cosine_sim)
    return fasttext_similarities

### Подсчёт косинусной близости пар слов из списков relatedness и similarity

In [11]:
fasttext_relatedness = calculate_fasttext_pairwise_cos_sims(fasttext_model=ft_model,
                                                            word_pair_similarities=relatedness_triples)
fasttext_similarities = calculate_fasttext_pairwise_cos_sims(fasttext_model=ft_model,
                                                            word_pair_similarities=similarity_triples)

### Подсчёт коэффициента корреляции Спирмена между человеческими оценками и косинусной близостью векторов, а также статистической значимости (p-value)

In [12]:
spearman_corr_relatedness = spearmanr(human_relatedness, fasttext_relatedness)
spearman_corr_similarities = spearmanr(human_similarities, fasttext_similarities)

relatedness_corr_coef, relatedness_corr_pvalue = spearman_corr_relatedness.correlation, spearman_corr_relatedness.pvalue
similarity_corr_coef, similarity_corr_pvalue = spearman_corr_similarities.correlation, spearman_corr_similarities.pvalue

print(f"Relatedness: Коэффициент корреляции Спирмена: {relatedness_corr_coef}, p-value: {relatedness_corr_pvalue}")
print(f"Similarity: Коэффициент корреляции Спирмена: {similarity_corr_coef}, p-value: {similarity_corr_pvalue}")

Relatedness: Коэффициент корреляции Спирмена: 0.7309719110157804, p-value: 2.228956668081054e-43
Similarity: Коэффициент корреляции Спирмена: 0.8318571698290462, p-value: 2.6920746558561095e-53
